# RNN

Ok, so I'm gonna try to make an RNN learn to speak like Hofstadter, trying to Google as little as possible :P

In [1]:
# LOAD GEBB
f = open("GEB.txt","r")
lines = list(f)

In [6]:
lines = list(filter(lambda x: x!= "\n",lines))

In [7]:
lines[:10]

['Contents \n',
 'Overview viii \n',
 'List of Illustrations xiv \n',
 'Words of Thanks xix \n',
 'Part I: GEB \n',
 'Introduction: A Musico-Logical Offering 3 \n',
 'Three-Part Invention 29 \n',
 'Chapter I: The MU-puzzle 33 \n',
 'Two-Part Invention 43 \n',
 'Chapter II: Meaning and Form in Mathematics 46 \n']

In [12]:
# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'-"
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [13]:
lines = [unicodeToAscii(l) for l in lines]

In [14]:
lines[:10]

['Contents ',
 'Overview viii ',
 'List of Illustrations xiv ',
 'Words of Thanks xix ',
 'Part I GEB ',
 'Introduction A Musico-Logical Offering  ',
 'Three-Part Invention  ',
 'Chapter I The MU-puzzle  ',
 'Two-Part Invention  ',
 'Chapter II Meaning and Form in Mathematics  ']

Ok, how do I make an RNN? Hmm

In [5]:
class RNN(nn.Module):
    def __init__()